In [ ]:
%load_ext autoreload
%autoreload 2

from train import *
from models import *
from tools import *

from keras import optimizers
from keras.models import load_model
from keras.utils import to_categorical
from keras.callbacks import *
from keras.preprocessing.image import ImageDataGenerator

from PIL import Image
from IPython.display import display

import numpy as np
import os
import random

In [ ]:
X_train, y_train = load_data()
X_val, y_val = load_data("val")

input_shape = X_train[0].shape
print("Input shape: " + str(input_shape))
print(X_train[0][0][0])

y_train = to_categorical(y_train, 200)
y_val = to_categorical(y_val, 200)

In [ ]:
labels = get_label_dict()
words = get_word_labels()

In [ ]:
# Image preprocessing for the data
datagen = ImageDataGenerator(horizontal_flip=True, 
                             shear_range=20., 
                             zoom_range=0.4, 
                             width_shift_range=0.2,
                             height_shift_range=0.2, 
                             channel_shift_range=0.1, 
                             fill_mode="reflect", 
                             preprocessing_function=shuffle_channels)

testgen = ImageDataGenerator()

data = {"train": (X_train, y_train), 
        "val": (X_val, y_val)}

In [ ]:
def lr_tune(optim_class, trials):
    results = []
    
    for _ in range(trials):
        lr = random.uniform(8e-4, 8e-3)
        optim = optim_class(learning_rate=lr)
        
        history = train(model_path=model_path, 
                        restore=False, 
                        epochs=5,
                        model=model, 
                        optim=optim,
                        datagen=datagen, 
                        testgen=testgen, 
                        data=data, 
                        cb_list=[], 
                        batch_size=32, 
                        verbosity=0)

        history = history.history

        train_acc = history["accuracy"][-1]
        val_acc = history["val_accuracy"][-1]

        result = "lr: {}, train: {}, val: {}\n".format(lr, train_acc, val_acc)
        results.append(result)
        print(result)
        
    return results

In [ ]:
keras.backend.clear_session()

model = AlphaNet(input_shape, "v5", quiet=True)
model_prefix = "models/" + model.name + "/" + model.name
model_path = model_prefix + ".h5"

In [ ]:
optim_list = [
    optimizers.Adam, 
    optimizers.RMSprop, 
    optimizers.Adadelta, 
    optimizers.SGD
]

for optim_class in optim_list:
    optim_name = str(optim_class)[25:-2]
    print(optim_name)
    results = lr_tune(optim_class, 3)
    file = open("tuning/" + model.name + "/" + optim_name + ".txt", "w")
    
    for result in results:
        file.write(result)
    
    file.close()